In [15]:
!git clone https://github.com/odashi/small_parallel_enja.git

fatal: destination path 'small_parallel_enja' already exists and is not an empty directory.


In [75]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB3
import scipy.stats as stats
import cv2 #im_rgb = cv2.cvtColor(im_cv, cv2.COLOR_BGR2RGB)
from sklearn.utils import class_weight
import json
import re
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import BaggingClassifier
import copy
import heapq

In [17]:
ls small_parallel_enja/

dev.en     train.en.000       train.en.vocab.all  train.ja.004
dev.ja     train.en.001       train.ja            train.ja.vocab.4k
README.md  train.en.002       train.ja.000        train.ja.vocab.all
test.en    train.en.003       train.ja.001
test.ja    train.en.004       train.ja.002
train.en   train.en.vocab.4k  train.ja.003


In [18]:
def Preprocess(text):
  text = "<start> "+text+" <end>"
  text = text.replace("\n", " ")
  text = re.sub(r"[' ']+", " ", text)
  return text

def PreprocessForTest(text, tokenizer):
  text = "<start> "+text+" <end>"
  text = text.replace("\n", " ")
  text = re.sub(r"[' ']+", " ", text)
  return text


def Read(filename):
  new_lines = []
  with open(filename, "r") as f:
    lines = f.readlines()
  for line in lines:
    new_lines.append(Preprocess(line))
  return new_lines
  
def BuildTokenizer(texts):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='\n', oov_token="<unk>")
  tokenizer.fit_on_texts(texts)
  return tokenizer

def Texts2Sequences(tok, train_texts, valid_texts, test_texts):
  train_sequences = tok.texts_to_sequences(train_texts)
  valid_sequences = tok.texts_to_sequences(valid_texts)
  test_sequences = tok.texts_to_sequences(test_texts)
  
  padded_train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding="post")

  return padded_train_sequences, valid_sequences, test_sequences 

In [19]:
DIR_NAME = "small_parallel_enja/"
train_ja = Read(DIR_NAME+"train.ja")
train_en = Read(DIR_NAME+"train.en")

valid_ja = Read(DIR_NAME+"dev.ja")
valid_en = Read(DIR_NAME+"dev.en")

test_ja = Read(DIR_NAME+"test.ja")
test_en = Read(DIR_NAME+"train.en")

In [20]:
ja_tokenizer = BuildTokenizer(train_ja)
en_tokenizer = BuildTokenizer(train_en)

In [21]:
train_ja_sequences, valid_ja_sequences, test_ja_sequences = Texts2Sequences(ja_tokenizer, train_ja, valid_ja, test_ja)
train_en_sequences, valid_en_sequences, test_en_sequences = Texts2Sequences(en_tokenizer, train_en, valid_en, test_en)

In [22]:
def CreateEncoderModel(vocab_size, units=512):
  emb_layer = tf.keras.layers.Embedding(vocab_size, units, mask_zero=True, name="encdoder_emb")#padding有効にするために、mask_zero=True
  gru_layer  = tf.keras.layers.GRU(units, name="encdoder_gru")
  encoder_inputs = tf.keras.Input(shape=(None,), name="encdoder_inputs")
  outputs = emb_layer(encoder_inputs)
  outputs = gru_layer(outputs)
  
  encoder = tf.keras.Model(encoder_inputs, outputs)

  return encoder

In [31]:
def CreateDecoderModel(vocab_size, units=512):

  emb_layer = tf.keras.layers.Embedding(vocab_size, units, mask_zero=True, name="decdoder_emb")#padding有効にするために、mask_zero=True
  gru_layer  = tf.keras.layers.GRU(units, return_sequences=True, name="decdoder_gru")
  dense_layer = tf.keras.layers.Dense(vocab_size, name="decdoder_outputs")

  decoder_inputs  = tf.keras.Input(shape=(None,), name="decdoder_inputs")
  decoder_hidden_states = tf.keras.Input(shape=(None,), name="decoder_hidden_states")

  outputs = emb_layer(decoder_inputs)
  gru_outputs = gru_layer(outputs, initial_state=decoder_hidden_states)
  outputs = dense_layer(gru_outputs)

  decoder = tf.keras.Model([decoder_inputs, decoder_hidden_states], [outputs, gru_outputs])

  return decoder

In [32]:
#評価方法はBLEU
def Evaluate(ja_texts, en_texts, encoder, decoder):
  score = 0.0
  for ja_text, en_text in zip(ja_texts, en_texts):
    predicted_text = Predict(ja_text, encoder, decoder)
    score += sentence_bleu([en_text.strip().split()], predicted_text.strip().split(), smoothing_function=SmoothingFunction().method4)
  return score/len(ja_texts)

In [127]:
def Predict(ja_text, encoder, decoder):
  #日本語文章が形態素解析されていない状態の場合は、MeCabを使ってください。
  #MeCabの使い方
  #m = MeCab.Tagger("-Owakati")
  #ja_text           = m.parse(ja_text).strip()
  
  sequence         = ja_tokenizer.texts_to_sequences([ja_text.strip()])
  sequence         = np.array(sequence)
  encoder_outputs = encoder(sequence)

  decoder_hidden_states = encoder_outputs
  decoder_inputs = np.array(en_tokenizer.texts_to_sequences([["<start>"]]))
  #20回で打ち切り
  #Decoderが出力する各時刻の最大確率をその時刻においての単語とする。(貪欲法)
  ans = "<start>"
  for _ in range(20):
    prediction, decoder_hidden_states = decoder([decoder_inputs, tf.reshape(decoder_hidden_states, [1, -1])])
    prediction_id = tf.argmax(prediction, axis=-1)[0][0].numpy()#貪欲法で単語を選択する
    word = en_tokenizer.sequences_to_texts([[prediction_id]])
    decoder_inputs = np.reshape(np.array(prediction_id), [-1, 1])
    ans += " "+word[0]
    if word[0]=="<end>":
      break
  return ans


In [34]:
class BestModel(tf.keras.callbacks.Callback):
    def __init__(self, bleu_function, valid_ja, valid_en, encoder, decoder):
        super(BestModel, self).__init__()
        #名前をロードする
        self.bleu_function = bleu_function
        self.valid_ja = valid_ja
        self.valid_en = valid_en
        self.encoder = encoder
        self.decoder = decoder
        self.best = 0

    def on_epoch_end(self, epoch, logs=None):
        print("validに対するBLEU")
        if self.best==0:
            self.model.save("best_model.h5")
        self.encoder.load_weights("best_model.h5", by_name=True)
        self.decoder.load_weights("best_model.h5", by_name=True)
        cur_bleu = self.bleu_function(self.valid_ja, self.valid_en, self.encoder, self.decoder)
        print(cur_bleu)
        if self.best < cur_bleu:
            self.best = cur_bleu
            self.best_weights = self.model.get_weights()
            self.model.save("best_model.h5")

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)


class CustomLearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self):
        super(CustomLearningRateScheduler, self).__init__()

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        print(self.model.optimizer.learning_rate)
        if epoch>=1 and epoch%5==0:
            lr = lr / 2
        
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)


In [48]:
BUFFER_SIZE = len(train_ja_sequences)
BATCH_SIZE  = 64
steps_per_epoch = len(train_en_sequences)//BATCH_SIZE
units = 512
vocab_jp_size = len(ja_tokenizer.word_index)+1
vocab_en_size = len(en_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((train_ja_sequences, train_en_sequences)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

encoder = CreateEncoderModel(vocab_jp_size)
decoder = CreateDecoderModel(vocab_en_size)

encoder.summary()
decoder.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encdoder_inputs (InputLayer) [(None, None)]            0         
_________________________________________________________________
encdoder_emb (Embedding)     (None, None, 512)         4494336   
_________________________________________________________________
encdoder_gru (GRU)           (None, 512)               1575936   
Total params: 6,070,272
Trainable params: 6,070,272
Non-trainable params: 0
_________________________________________________________________
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decdoder_inputs (InputLayer)    [(None, None)]       0                                            
_______________________________________________________

In [52]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask#mask処理を行なっている。0をかけることで、損失が0になり、誤差逆伝播時も微分値が０になる。

  return tf.reduce_mean(loss_)#バッチで割っている。時刻Tでは割っていないことに注意してください。

In [64]:
@tf.function
def train_step(inp, targ):
  global units

  loss = 0

  with tf.GradientTape() as tape:
    enc_output = encoder(inp)

    dec_hidden = enc_output

    # Teacher forcingと呼ばれる学習手法
    for t in range(0, targ.shape[1]-1):#<end>トークンは入力する必要がない。
      dec_inputs                        = tf.expand_dims(targ[:, t], 1)#時刻tの単語を入力する
    
      predictions, dec_hidden = decoder([dec_inputs, dec_hidden])
      dec_hidden = tf.reshape(dec_hidden, [-1, units])
      loss += loss_function(targ[:, t+1], predictions)#正解データは時刻t+1の単語

      # using teacher forcing
      dec_inputs = tf.expand_dims(targ[:, t+1], 1)
    batch_loss = (loss / int(targ.shape[1]))#時刻分だけ割る

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(batch_loss, variables)#微分値をゲット

  optimizer.apply_gradients(zip(gradients, variables))#更新する

  return batch_loss

In [65]:
EPOCHS = 10

for epoch in range(EPOCHS):
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ)
    total_loss += batch_loss
    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  #本来はここで、検証データに対する精度を求める必要があるが、略。
  #バッチごとに、検証データに対する精度を求める必要はない。あんまり細かくチェックしても、意味がないからです。
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'loss' is nonlocal and global (tmpvcy30r_4.py, line 20)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'loss' is nonlocal and global (tmpvcy30r_4.py, line 20)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1 Batch 0 Loss 1.9829
Epoch 1 Batch 100 Loss 2.0747
Epoch 1 Batch 200 Loss 2.2108
Epoch 1 Batch 300 Loss 1.8194
Epoch 1 Batch 400 Loss 1.9228
Epoch 1 Batch 500 Loss 1.9594
Epoch 1 Batch 600 Loss 1.8879
Epoch 1 Batch 700 Loss 1.9292
Epoch 1 Loss 1.9835
Epoch 2 Batch 0 Loss 1.8692
Epoch 2 Batch 100 Loss 1.8887
Epoch 2 Batch 200 Loss 1.8158
Epoch 2 Ba

In [67]:
encoder.save("encoder.h5")
decoder.save("decoder.h5")

In [304]:
Evaluate(valid_ja, valid_en, encoder, decoder)

0.3380265875041033

In [72]:
Evaluate(test_ja, test_en, encoder, decoder)

0.19309505176539324

In [297]:
#Beam Search

def BeamSearch(ja_text, beam_width):
    sequence         = ja_tokenizer.texts_to_sequences([ja_text.strip()])
    sequence         = np.array(sequence)
    encoder_outputs = encoder(sequence)

    decoder_hidden_states = encoder_outputs
    decoder_inputs = np.array(en_tokenizer.texts_to_sequences([["<start>"]]))


    #初期状態
    list_for_heap = []#ヒープ用のリスト

    ans = []

    top_k = []

    #<start>だけ特別
    probability_distribution, decoder_hidden_states = decoder([decoder_inputs, tf.reshape(decoder_hidden_states, [1, -1])])
    for word_probability, word_id in zip(*tf.nn.top_k(tf.nn.softmax(probability_distribution)[0][0], beam_width)):
        word     = en_tokenizer.sequences_to_texts([[word_id.numpy()]])[0]
        element = (word_probability*-1, word,  "<start>"+ " " + word, decoder_hidden_states)
        heapq.heappush(list_for_heap, element)

    for _ in range(20):
        #上位3文章をここで、決定する。
        for __ in range(beam_width):
            
            top_k.append(heapq.heappop(list_for_heap))

        list_for_heap = []#heap用のリストを初期化する

        for i in range(beam_width):
            sentence_node = top_k[i]
            pre_prob = sentence_node[0]*-1
            pre_word = sentence_node[1]
            sentence = sentence_node[2]
            decoder_hidden_states = sentence_node[3]
            decoder_inputs = np.reshape(np.array(en_tokenizer.word_index[pre_word]), [-1, 1])
            if pre_word=="<end>":#<end>トークンならば、それ以上の処理をする必要がない
                ans.append((pre_prob, sentence))
                continue

            probability_distribution, decoder_hidden_states = decoder([decoder_inputs, tf.reshape(decoder_hidden_states, [1, -1])])
            for word_probability, word_id in zip(*tf.nn.top_k(tf.nn.softmax(probability_distribution)[0][0], beam_width)):
                word     = en_tokenizer.sequences_to_texts([[word_id.numpy()]])[0]
                node = (word_probability.numpy() * pre_prob*-1, word,  sentence + " " + word, decoder_hidden_states)
                heapq.heappush(list_for_heap, node)

        top_k = []
        if len(ans)>=beam_width:#
            break

    ans = sorted(ans, key=lambda x:x[0], reverse=True)
    return ans[0][1]

In [298]:
beam_width = 3


for index in range(len(valid_ja)):
    print("target : ")
    print(valid_en[index])

    print("")

    print("greedy search : ")
    print(Predict(valid_ja[index], encoder, decoder))

    print("")

    print("bearm search : ")
    print(BeamSearch(valid_ja[index], beam_width))
    print("")

target : 
<start> show your own business . <end>

greedy search : 
<start> do your own heart . <end>

bearm search : 
<start> do your best . <end>

target : 
<start> he lived a hard life . <end>

greedy search : 
<start> he lived a moral life . <end>

bearm search : 
<start> he lived a moral life . <end>

target : 
<start> no . i m sorry , i ve got to go back early . <end>

greedy search : 
<start> sorry , i m sorry . i ll have to go back . <end>

bearm search : 
<start> i m sorry , i can t come . <end>

target : 
<start> she wrote to me to come at once . <end>

greedy search : 
<start> she said she would come back to me . <end>

bearm search : 
<start> she said she would come back to me . <end>

target : 
<start> i can t swim at all . <end>

greedy search : 
<start> i can t swim any more . <end>

bearm search : 
<start> i can t swim any more . <end>

target : 
<start> is there any hope of his success ? <end>

greedy search : 
<start> do you have his own way in life ? <end>

bearm sear

KeyboardInterrupt: ignored

In [252]:
BeamSearch(valid_ja[41])

where
which
how
when
what
<start> where is the bathroom ? <end>
<start> where is the cafeteria ? <end>
<start> where can i find it ? <end>
<start> where s the way there ? <end>


'<start> where is the bathroom ? <end>'

In [306]:
#評価方法はBLEU
def EvaluateWithBeamSearch(ja_texts, en_texts, encoder, decoder, beam_width):
  score = 0.0
  for ja_text, en_text in zip(ja_texts, en_texts):
    predicted_text = BeamSearch(ja_text, beam_width)
    score += sentence_bleu([en_text.strip().split()], predicted_text.strip().split(), smoothing_function=SmoothingFunction().method4)
  return score/len(ja_texts)

In [ ]:
EvaluateWithBeamSearch(valid_ja, valid_en, encoder, decoder, 10)

In [229]:
#beam_width : validに対するBLEU 34

<tf.Tensor: shape=(1, 1, 6635), dtype=float32, numpy=
array([[[1.3077504e-07, 2.7298949e-12, 2.6554095e-12, ...,
         2.9605807e-10, 3.6966572e-11, 2.9319003e-13]]], dtype=float32)>